In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

> ### Importing Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

> ### Loading Files

In [ ]:
%%time
train_df=pd.read_csv('../input/santander-customer-satisfaction/train.csv').set_index('ID')
test_df=pd.read_csv('../input/santander-customer-satisfaction/test.csv').set_index('ID')

> ### Data Exploration

In [ ]:
def reduce_mem_usage(df):
    """ 
    iterate through all the columns of a dataframe and 
    modify the data type to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage of dataframe is {:.2f}''MB').format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max <np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max <np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max <np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max <np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                #if c_min > np.finfo(np.float16).min and c_max <np.finfo(np.float16).max:
                    #df[col] = df[col].astype(np.float16)
                if c_min > np.finfo(np.float32).min and c_max <np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage after optimization is: {:.2f}''MB').format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
#Reducing memory
train_df=reduce_mem_usage(train_df)

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
#Custom Code to Display properties of each column
l=[]
for i in train_df.columns:
    l.append([i,len(train_df[i].unique()),max(train_df[i].unique()),min(train_df[i].unique()),train_df[i].var(),train_df[i].astype(bool).sum(axis=0),train_df[i].count(),list(train_df[i].unique())])
ldf=pd.DataFrame(l, columns=['Features', 'No_Unique_Values', 'Max_Value','Min_Value','Variance','Non-Zero','Total_Values','Unique_Values'])
ldf.head(307)

In [ ]:
print (sorted(ldf['No_Unique_Values'].unique()))

In [ ]:
ldf[ldf['Non-Zero']<10000]

In [ ]:
del_features=list(ldf[ldf.No_Unique_Values==1].Features)
ldf[ldf.No_Unique_Values==1].shape
#Remove these Columns : No Unique Value - 34 Columns

In [ ]:
max_impute=list(ldf[ldf.Max_Value==9999999999].Features)
for i in max_impute:
    if i in train_df.columns:
        print (train_df[train_df[i]==9999999999].shape,'\t',i)
    else:
        print ('Column Removed')
#missing values in max_impute Columns - 307 rows each - Impute the values - Use median
#max_impute

In [ ]:
min_impute=list(ldf[ldf.Min_Value==-999999.00].Features)
for i in min_impute:
    if i in train_df.columns:
        print (train_df[train_df[i]==-999999.00].shape,'\t',i)
    else:
        print ('Column Removed')
#missing values in max_impute Columns - 307 rows each - Impute the values - Use median
#min_impute

In [ ]:
#replacing missing values
train_df.replace({9999999999:np.NaN,-999999:np.NaN},inplace=True)
train_df.fillna(train_df.median(), inplace=True)

In [ ]:
del_ID=list(train_df[train_df.duplicated()==True].index)
train_df[train_df.duplicated()==True].shape
#Duplicated Records - To be removed - 4807 rows

In [ ]:
columns=list(train_df.columns)

In [ ]:
fea_del_set=set()
for i in range(len(columns)):
    for j in range(i+1,len(columns)):
        if train_df[columns[i]].equals(train_df[columns[j]]) and columns[j] not in fea_del_set:
            fea_del_set.add(columns[j])
#62 Columns        

In [ ]:
##Double Check
"""fea_del={}
for i in range(len(columns)):
    l=[]
    for j in range(i+1,len(columns)):
        if train_df[columns[i]].equals(train_df[columns[j]]):
            l.append(columns[j])
    if l!=[]:
        fea_del[columns[i]]=l
##Check accuracy
set1=set()
for i in list(fea_del.values()):
    for j in i:
        set1.add(j)
assert len(set1)==len(fea_del_set)"""
pass       

In [ ]:
del_col=fea_del_set.union(del_features)
#63 columns

In [ ]:
#del_col
"""
{'delta_num_reemb_var13_1y3',
 'delta_num_reemb_var17_1y3',
 'delta_num_reemb_var33_1y3',
 'delta_num_trasp_var17_in_1y3',
 'delta_num_trasp_var17_out_1y3',
 'delta_num_trasp_var33_in_1y3',
 'delta_num_trasp_var33_out_1y3',
 'imp_amort_var18_hace3',
 'imp_amort_var34_hace3',
 'imp_reemb_var13_hace3',
 'imp_reemb_var33_hace3',
 'imp_trasp_var17_out_hace3',
 'imp_trasp_var33_out_hace3',
 'ind_var13_medio',
 'ind_var18',
 'ind_var2',
 'ind_var25',
 'ind_var26',
 'ind_var27',
 'ind_var27_0',
 'ind_var28',
 'ind_var28_0',
 'ind_var29',
 'ind_var29_0',
 'ind_var2_0',
 'ind_var32',
 'ind_var34',
 'ind_var37',
 'ind_var39',
 'ind_var41',
 'ind_var46',
 'ind_var46_0',
 'num_reemb_var13_hace3',
 'num_reemb_var33_hace3',
 'num_trasp_var17_out_hace3',
 'num_trasp_var33_out_hace3',
 'num_var13_medio',
 'num_var18',
 'num_var25',
 'num_var26',
 'num_var27',
 'num_var27_0',
 'num_var28',
 'num_var28_0',
 'num_var29',
 'num_var29_0',
 'num_var2_0_ult1',
 'num_var2_ult1',
 'num_var32',
 'num_var34',
 'num_var37',
 'num_var39',
 'num_var41',
 'num_var46',
 'num_var46_0',
 'saldo_medio_var13_medio_hace3',
 'saldo_medio_var13_medio_ult1',
 'saldo_var27',
 'saldo_var28',
 'saldo_var29',
 'saldo_var2_ult1',
 'saldo_var41',
 'saldo_var46'}
"""
len(del_col)

In [ ]:
train_df.drop(del_col, axis=1,inplace=True)

In [ ]:
train_df.drop_duplicates(keep='first',inplace=True)

In [ ]:
train_df.tail()

In [ ]:
train_df.shape

In [ ]:
f_u2=list(ldf[ldf.No_Unique_Values==2].Features)
f_u3=list(ldf[ldf.No_Unique_Values==3].Features)

In [ ]:
def unique_percent(df,column):
    unique=list(df[column].unique())
    total=df[column].count()
    count=[]
    percent=[]
    for i in unique:
        count.append((i,(df[column]==i).sum()))
        percent.append((i,(df[column]==i).sum()/total*100))
    return count,percent

In [ ]:
count=[]
percent=[]
for i in f_u2:
    c,p=unique_percent(train_df,i)
    count.append((i,c))
    percent.append((i,p))
count

In [ ]:
count=[]
percent=[]
for i in f_u3:
    c,p=unique_percent(train_df,i)
    count.append((i,c))
    percent.append((i,p))
percent

In [ ]:
train_df[train_df.num_reemb_var17_hace3==3]